# MIMIC-EYE 1.0 - An Eye-Tracking Data Analytics Toolkit for REFLACX and EYE GAZE

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np
import os
import pickle as pkl
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from mimic.Patient import Patient
from mimic.Xray import Xray
from MIMIC_EYE import MIMIC_EYE 
from Constants import Constants as c
from data.groundtruth.BoundingBox import BoundingBox

from util.mimic_data_processing import *

In [3]:
# starting program
mimic_eye = MIMIC_EYE()

In [4]:
c.CACHE = {
						"REFLACX" : 0,							# counts how many patients are in REFLACX dataset
						"EYE_GAZE" : 0,							# counts how many patients are in EYE GAZE dataset
						"OVERLAP" : [],							# lists the patient ids that overlap both datasets
						"PATIENTS" : {},						# dictionary containing all information about a single patient
						"XRAY_TO_PATIENT" 	: {},		# dictionary mapping an XRayID to a PatientID
						"XRAY_TO_DIAGNOSIS" : {},		# dictionary mapping an XRayID to a Diagnosis (Eye Gaze only)
						"XRAY_TO_ABNORMALITY" : {},	# dictionary mapping an XRayID to an Abnormality (REFLACX only)
	}

In [5]:
PATIENTS_DIC = {}
XRAY_TO_PATIENT = {}
XRAY_TO_DIAGNOSIS = {}
XRAY_TO_ABNORMALITY = {}

In [6]:
# check which column ("Normal", "CHF" or "Pneumonia") has a 1 in the eye_gaze_df dataframe
def get_diagnosis(row):
    diagnosis = "Normal"
    if row.CHF == 1:
        diagnosis = "CHF"
    if row.pneumonia == 1:
        diagnosis = "Pneumonia"
    return diagnosis


In [8]:
eye_gaze_sheet = os.path.join(c.DATASET_PATH, "spreadsheets", 'EyeGaze', "master_sheet_with_updated_stayId.csv")
eye_gaze_df = pd.read_csv( eye_gaze_sheet )

# extract all patient keys from EYE GAZE dataset
patient_keys = eye_gaze_df['patient_id'].values.tolist()
for patient_key in patient_keys:

  # check if a patient key exists in PATIENTS_DIC
  if patient_key in PATIENTS_DIC:
    temp = eye_gaze_df[ eye_gaze_df['patient_id'] == patient_key ]
    print(f"Patient {patient_key} already exists in PATIENTS_DIC. Processing DICOM ID {temp.dicom_id.values[0]}")
    print(f"Current DICOM IDs {PATIENTS_DIC[patient_key].get_xray_ids()}")
    print("---------------------------------------------------------------------\n")
  else:
    c.CACHE["EYE_GAZE"] = c.CACHE["EYE_GAZE"] + 1

    patient_dirs = os.listdir(c.PATIENT_PATH(c.DATASET_PATH, patient_key))
    if "REFLACX" in patient_dirs:
      c.CACHE["REFLACX"] = c.CACHE["REFLACX"] + 1
      c.CACHE["OVERLAP"].append(patient_key)

    # performs an inner join between the eye_gaze dataframe and the MIMIC core data on the patient_id key
    # this allows us to obtain all information about a patient
    # if there are multiple records of the patient (due to several visits to the hospital), we record the 
    # patient's information of his last visit to the hospital
    mimic_core = join_patients_mimic_core_data( patient_key, eye_gaze_df)

    # create a new Patient
    patient = Patient(mimic_core["patient_id"].tail(1).values[0], \
                      mimic_core["gender_x"].tail(1).values[0], \
                      mimic_core["language"].tail(1).values[0], \
                      mimic_core["marital_status"].tail(1).values[0],\
                      mimic_core["race"].tail(1).values[0], \
                      mimic_core["anchor_age_x"].tail(1).values[0], \
                      mimic_core["anchor_year"].tail(1).values[0], \
                      mimic_core["dod"].tail(1).values[0], \
                      mimic_core["anchor_year_group"].tail(1).values[0], {}, {}  )

    # get all CXR images associated to patient with patient_key
    xray_lst = []
    patient_df = eye_gaze_df[ eye_gaze_df[ "patient_id"] == patient_key ]
    for row in patient_df.itertuples():
      xray_id = row.dicom_id
      study_id = "s"+str(row.study_id)
      report = read_CXR_report( patient_key, study_id )
      diagnosis = get_diagnosis(row)
    
      XRAY_TO_DIAGNOSIS[xray_id] = diagnosis

      # a patient can have many imagaeology studies, and each study is composed of at least one XRay image
      # one xray is given by an image file, a textual report with all the findings and diagnosis
      # and a set of 
      xray_dicom_path = c.MIMIC_DICOM_PATH(c.DATASET_PATH, patient_key, study_id, xray_id)
    
      annotation_lst = []
      bboxes_df = pd.read_csv( c.EYE_GAZE_BBOX_PATH(c.DATASET_PATH, patient_key) )
      for bbox in bboxes_df.itertuples():
        bbox = BoundingBox(bbox.x1, bbox.x2, bbox.y1, bbox.y2, bbox.bbox_name)
        annotation_lst.append( bbox )
        
      xray_lst.append( Xray(xray_id, study_id, report, diagnosis, xray_dicom_path, annotation_lst))
      XRAY_TO_PATIENT[xray_id] = patient

    patient.setXray_lst(xray_lst)
    PATIENTS_DIC[patient_key] = patient
    
  

c.CACHE["PATIENTS"] = PATIENTS_DIC
c.CACHE["XRAY_TO_PATIENT"] = XRAY_TO_PATIENT
c.CACHE["XRAY_TO_DIAGNOSIS"] = XRAY_TO_DIAGNOSIS
c.CACHE["XRAY_TO_ABNORMALITY"] = XRAY_TO_ABNORMALITY
c.CACHE["OVERLAP"] = list(set(c.CACHE["OVERLAP"]))

    

    
    

  

  # find all Xrays belonging to patient with key patient_id
  
  #xrayStudy = XrayStudy(  )


  # create a new Xray
  
  


Patient 15628804 already exists in PATIENTS_DIC. Processing DICOM ID 24c7496c-d7635dfe-b8e0b87f-d818affc-78ff7cf4
Current DICOM IDs []
Patient 19462352 already exists in PATIENTS_DIC. Processing DICOM ID 78711a04-264d5305-d5feec9b-ebef1cec-fdc6db9c
Current DICOM IDs []
Patient 11255143 already exists in PATIENTS_DIC. Processing DICOM ID a770d8d6-7b6a62ff-815ab876-c81709a8-9a654a54
Current DICOM IDs []
Patient 10526322 already exists in PATIENTS_DIC. Processing DICOM ID 8e457921-bc1af8aa-a65073c1-aaac8247-c5ceb780
Current DICOM IDs []
Patient 12055181 already exists in PATIENTS_DIC. Processing DICOM ID 62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a
Current DICOM IDs []
Patient 13511876 already exists in PATIENTS_DIC. Processing DICOM ID 64beda42-873c8a44-3d9d3073-33372d32-dee84707
Current DICOM IDs []
Patient 12661994 already exists in PATIENTS_DIC. Processing DICOM ID 748991dc-fb330cd1-fc203d00-b89b5ccb-7995b475
Current DICOM IDs []
Patient 17243626 already exists in PATIENTS_DIC. Proces

In [8]:
c.CACHE

{'REFLACX': 10,
 'EYE_GAZE': 147,
 'OVERLAP': [12658584,
  19584206,
  14150037,
  17023312,
  13473495,
  15225006,
  11537996,
  19856485,
  18426683,
  15752803],
 'PATIENTS': {},
 'XRAY_TO_PATIENT': {},
 'XRAY_TO_DIAGNOSIS': {},
 'XRAY_TO_ABNORMALITY': {}}

In [12]:
eye_gaze_df[ eye_gaze_df["dicom_id"] == "62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a"]

,dicom_id,path,study_id,patient_id,stay_id,gender,anchor_age,image_top_pad,image_bottom_pad,image_left_pad,image_right_pad,dx1,dx1_icd,dx2,dx2_icd,dx3,dx3_icd,dx4,dx4_icd,dx5,dx5_icd,dx6,dx6_icd,dx7,dx7_icd,dx8,dx8_icd,dx9,dx9_icd,normal_reports,Normal,CHF,pneumonia,consolidation,enlarged_cardiac_silhouette,linear__patchy_atelectasis,lobar__segmental_collapse,not_otherwise_specified_opacity___pleural__parenchymal_opacity__,pleural_effusion_or_thickening,pulmonary_edema__hazy_opacity,normal_anatomically,elevated_hemidiaphragm,hyperaeration,vascular_redistribution,atelectasis__chx,cardiomegaly__chx,consolidation__chx,edema__chx,enlarged_cardiomediastinum__chx,fracture__chx,lung_lesion__chx,lung_opacity__chx,no_finding__chx,pleural_effusion__chx,pleural_other__chx,pneumonia__chx,pneumothorax__chx,support_devices__chx,cxr_exam_indication
4,62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a,files/p12/p12055181/s59722264/62fe5d5a-1806ee3...,59722264,12055181,37532814.0,M,20 - 30,0,0,534,534,"Heart failure, unspecified",I50.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,NaN,1.0,NaN,0.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,-1.0,NaN,NaN,"___M with DOE, SOB in supine position // eval..."


In [5]:
patient.getID()

15628804

In [13]:
sample.columns

Index(['Unnamed: 0_x', 'patient_id', 'gender_x', 'anchor_age_x', 'anchor_year',
       'anchor_year_group', 'dod', 'Unnamed: 0_y', 'hadm_id_x', 'admittime',
       'dischtime', 'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'marital_status', 'race',
       'edregtime', 'edouttime', 'hospital_expire_flag', 'Unnamed: 0',
       'hadm_id_y', 'transfer_id', 'eventtype', 'careunit', 'intime',
       'outtime', 'dicom_id', 'path', 'study_id', 'stay_id', 'gender_y',
       'anchor_age_y', 'image_top_pad', 'image_bottom_pad', 'image_left_pad',
       'image_right_pad', 'dx1', 'dx1_icd', 'dx2', 'dx2_icd', 'dx3', 'dx3_icd',
       'dx4', 'dx4_icd', 'dx5', 'dx5_icd', 'dx6', 'dx6_icd', 'dx7', 'dx7_icd',
       'dx8', 'dx8_icd', 'dx9', 'dx9_icd', 'normal_reports', 'Normal', 'CHF',
       'pneumonia', 'consolidation', 'enlarged_cardiac_silhouette',
       'linear__patchy_atelectasis', 'lobar__segmental_collapse',
       'not_otherwise_speci

# MANIM

In [4]:
from manim import *

from manim.scene.scene import Scene
from manim.mobject.types.image_mobject import ImageMobject
import manim_ml
import cv2 
config.media_embed = True

In [47]:
image_p = cv2.imread("image.jpg")
image_p.shape

(3056, 2544, 3)

In [16]:
%%manim -qm  -r 3056,2544 -v WARNING CreateCircle

class CreateCircle(Scene):
    def construct(self):
        image = ImageMobject(cv2.imread("image.jpg"))
        image.height = 11.5
        print(image.width)

        self.add(image)

        lst = [ np.array([0., 0., 0.]), np.array([0., 2., 0.]), np.array([2., 0., 0.]), np.array([2., 2., 0.]) ]
        for l in lst:
            print(l)
            circle = Dot(point=l, radius=0.03, stroke_width=1, fill_opacity=0.80, color='#C70039')
            
            self.play(Create(circle))


9.573298429319374
[0. 0. 0.]


[0. 2. 0.]


[2. 0. 0.]


[2. 2. 0.]


In [16]:
# Create an instance of the MyAnimation class
animation = MyAnimation()

# Call the construct method to run the animation
animation.construct()

In [14]:
%%manim -qm -v WARNING SquareToCircle

class SquareToCircle(Scene):
   def construct(self):
      square = Square()
      circle = Circle()
      circle.set_fill(PINK, opacity=0.5)
      self.play(Create(square))
      self.play(Transform(square, circle))
      self.wait()

In [6]:


animation.construct()

TypeError: animate_dot() missing 1 required positional argument: 'coordinates'